# Function calling with Anthropic Claude LLM and Amazon Bedrock

In this notebook we explore how to do function calling with Anthropic Claude LLMs on Amazon Bedrock.
To keep it simple, we use raw propmts to acheive function calling.

## Why function calling

For example, a user might want to ask Claude to get the weather forecast. Claude is unable to perform this action by itself, but when using with the function calling prompt template, Claude can decide to call a function named get_weather(location: str) that has been described in the prompt template.

Through the function calling prompt, customers can now describe functions to Claude and have the model intelligently choose to use the functions to answer user questions.


Anthropic has a Python API library [`anthropic-bedrock`](https://github.com/anthropics/anthropic-bedrock-python) specifically tailored for Bedrock. Refer to [anthropic-bedrock](https://github.com/anthropics/anthropic-bedrock-python) github repo for more info.

This notebook is tested with Bedrock modelId **`anthropic.claude-v2`**.

In [1]:
# Install all prerequisites to run this notebook
import json
import os
import re
import sys
import time
from types import FunctionType

import boto3
import tools
from anthropic_bedrock import AI_PROMPT, HUMAN_PROMPT, AnthropicBedrock
from IPython.display import Markdown
from loguru import logger
from rich import print
from rich.status import Status

!pip install -r requirements.txt --quiet

# load rich extension for pretty printing with format.
%load_ext rich

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

### Create Anthropic client

In [3]:
session = boto3.Session()
creds = session.get_credentials()

client = AnthropicBedrock(
    aws_access_key=creds.access_key,
    aws_secret_key=creds.secret_key,
    aws_session_token=creds.token,
    aws_region=session.region_name,
)

model_id = "anthropic.claude-v2"

### Helper functions

Here we define the following helper functions.

1. `create_prompt` function returns the function calling prompt with function definitions along with few-shot examples
2. `add_tools` function gets all function definitions in a XML formatted string
3. `execute_function` extracts function name and function args from LLM output, executes the function and returns the function output in `<function_result></function_result>` tags
4. `invoke_llm` invokes **`anthropic.claude-v2`** and returns `completion`, `stop_reason` and `stop_sequence`

In [4]:
def create_prompt(tools_string, user_input):
    template = f"""
Human: You are a research assistant AI that has been equipped with the following function(s) to help you answer a <question>. Your goal is to answer the user's question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. The result of a function call will be added to the conversation history as an observation.

Here are the only function(s) I have provided you with:

<functions> {tools_string} </functions>

Note that the function arguments have been listed in the order that they should be passed into the function.

Do not modify or extend the provided functions under any circumstances. For example, calling get_current_temp() with additional parameters would be considered modifying the function which is not allowed. Please use the functions only as defined.

DO NOT use any functions that I have not equipped you with.

To call a function, output <function_call>insert specific function</function_call>. You will receive a <function_result> in response to your call that contains information that you can use to better answer the question.

Here is an example of how you would correctly answer a question using a <function_call> and the corresponding <function_result>. Notice that you are free to think before deciding to make a <function_call> in the <scratchpad>:

<example>
<functions>
<function>
<function_name>get_current_temp</function_name>
<function_description>Gets the current temperature for a given city.</function_description>
<required_argument>city (str): The name of the city to get the temperature for.</required_argument>
<returns>int: The current temperature in degrees Fahrenheit.</returns>
<raises>ValueError: If city is not a valid city name.</raises>
<example_call>get_current_temp(city="New York")</example_call>
</function>
</functions>

<question>What is the current temperature in San Francisco?</question>

<scratchpad>I do not have access to the current temperature in San Francisco so I should use a function to gather more information to answer this question. I have been equipped with the function get_current_temp that gets the current temperature for a given city so I should use that to gather more information.

I have double checked and made sure that I have been provided the get_current_temp function. I have double checked that the <function_call> tags contain only the function call and nothing else.
</scratchpad>

<function_call>get_current_temp(city="San Francisco")</function_call>

<function_result>71</function_result>

<answer>The current temperature in San Francisco is 71 degrees Fahrenheit.</answer>
</example>

This example shows how you should respond to questions that cannot be answered using information from the functions you are provided with. Remember, DO NOT use any functions that I have not provided you with.

Remember, your goal is to answer the user's question to the best of your ability, using only the function(s) provided to gather more information if necessary to better answer the question.

Do not modify or extend the provided functions under any circumstances. For example, calling get_current_temp() with additional parameters would be modifying the function which is not allowed. Please use the functions only as defined.

The result of a function call will be added to the conversation history as an observation. If necessary, you can make multiple function calls and use all the functions I have equipped you with. Let's create a plan and then execute the plan. Double check your plan to make sure you don't call any functions that I haven't provided. Always return your final answer within  <answer></answer> tags.

DO NOT output any preamble like 'based on calling the provided functions ...'. If <function_result> contains items in a list or tuple then format them using markdown list. Do NOT include <function_result> tags in the output.
Just answer the <question> in a direct manner.

The question to answer is <question> {user_input} </question>


Assistant:
"""
    return template


def add_tools():
    tools_string = ""
    for tool_spec in tools.list_of_function_specs:
        tools_string += tool_spec
    return tools_string


def execute_function(func_text: str):
    def format_result(output):
        return f"""<function_result>{output}</function_result>"""

    func_name = re.search(r"([^\(]+)\(", func_text).group(1)
    func = getattr(tools, func_name)  # Get reference to function from mytools
    func_text = func_text.replace("()", "")
    kwargs = {}
    if "(" in func_text and ")" in func_text:
        args_str = re.search(r"\(([^\)]+)\)", func_text).group(1)
        for arg in args_str.split(","):
            parts = arg.split("=")  # Split by =
            key = parts[0].strip()  # Strip whitespace in param name
            value = parts[1]
            kwargs[key] = value

    # logger.info(f"Extracted function_name: {func_name}")
    # logger.info(f"Extracted args: {kwargs}")
    assert isinstance(func, FunctionType)  # Check that it is callable
    if len(kwargs) >= 1:
        result = format_result(
            func(**kwargs)
        )  # Call the function and format the result
    else:
        result = format_result(func())
    return result


def get_function_result(partial_completion):
    """extracts function name and arguments, executes the func"""
    # LLM outputs <function_call>get_lat_long(place="NYC")
    # We extract text after <function_call> and call execute_function
    start_index = partial_completion.find("<function_call>")
    function_result = ""
    if start_index != -1:
        extracted_text = partial_completion[start_index + 15 :].strip()
        # logger.info(f"Extracted Text: {extracted_text}")
        function_result = execute_function(extracted_text)
    return function_result


def invoke_llm(
    client,
    prompt: str,
    modelId: str = model_id,
    max_tokens: int = 1000,
    temperature: float = 0.0,
):
    try:
        partial_completion = client.completions.create(
            prompt=prompt,
            stop_sequences=["\n\nHuman:", "</function_call>"],
            model=modelId,
            max_tokens_to_sample=max_tokens,
            temperature=temperature,
        )
        return (
            partial_completion.completion,
            partial_completion.stop_reason,
            partial_completion.stop,
        )
    except anthropic_bedrock.APIConnectionError as e:
        logger.error("The server could not be reached")
        logger.error(
            e.__cause__
        )  # an underlying Exception, likely raised within httpx.
    except anthropic_bedrock.RateLimitError as e:
        logger.error("A 429 status code was received; we should back off a bit.")
    except anthropic_bedrock.APIStatusError as e:
        logger.error("Another non-200-range status code was received")
        logger.error(e.status_code)
        logger.error(e.response)

### Create prompt

1. Enter question to be answered
2. get the list of function definitions (`add_tools` function)
3. call `create_prompt` with functions

>**NOTE:** [tools.py](./tools.py) currently defines 3 functions but has function definitions only for 2. Because `get_weather` calls `get_weather_code` function. `get_weather_code` is not provided as a function definition. Refer to [tools.py](./tools.py) for more info.

### Invoke LLM in a loop until desired `stop_sequence`

To invoke `Claudev2`, we call the `invoke_llm` function. This function returns 3 values:
- `partial_completion` (output from the LLM)
- `stop_reason`
- `stop_sequence`
   - a new stop sequence called `</function_call>` is added when calling `client.completions.create` in `invoke_llm` function. We use this `stop_sequence` to determine when to stop invoking the LLM.
 
#### Invoke LLM in a loop until `HUMAN_PROMPT`

Here's what's happening below:
1. From the output find string `<function_call>` and strip this text (`extracted_text`)
2. Call `execute_function` with `extracted_text`
   - `execute_function` returns output formatted in `<function_result>` tags
3. We now, need to reformat the original prompt with the `function_result` and add `AI_PROMPT` i.e `\n\nAssistant:` to the end of the prompt.
4. call `invoke_llm` again and repeat this until `stop_sequence` matches `HUMAN_PROMPT` i.e., `\n\nHuman:`, this marks the end of the loop.
5. Finally, print the output.

In [5]:
from rich.console import Console

console = Console()

question = "What is the weather in Gainesville"  # Question we need answer for
function_defs = add_tools()  # Get list of function definitions
prompt = create_prompt(function_defs, question)  # Get prompt

with console.status("[magenta]Invoking LLM...") as status:
    token_count = client.count_tokens(prompt)  # Keep track of total tokens
    status.update(status="[bold blue] Invoking LLM ...", spinner="earth")
    partial_completion, stop_reason, stop_seq = invoke_llm(client, prompt)
    token_count = client.count_tokens(prompt + partial_completion)
    status.update(
        status=f"[bold blue] Invoking LLM Token count: {token_count} ...",
        spinner="earth",
    )
    while stop_seq == "</function_call>":
        start_index = partial_completion.find("<function_call>")
        if start_index != -1:
            extracted_text = partial_completion[start_index + 15 :].strip()
            status.update(
                status=f"executing function: [green]{extracted_text}[/green]",
                spinner="earth",
            )
            function_result = execute_function(extracted_text)
            # logger.info(f"function: {extracted_text} result: {function_result}")
            prompt += f"{partial_completion}{stop_seq}\n{function_result}{AI_PROMPT}"
            partial_completion, stop_reason, stop_seq = invoke_llm(client, prompt)
            token_count += client.count_tokens(prompt + partial_completion)

if stop_seq == HUMAN_PROMPT:
    status.stop()
    logger.info(f"Total Tokens: {token_count}")
    print(f"[green]Question:[/green] [b]{question}[/b]\n")
    print(f"{partial_completion}")

Output()

2023-11-25 12:20:08.443 | INFO     | tools:get_weather:94 - {'temperature': 69.38150024414062, 'conditions': 'Clear sky'}


2023-11-25 12:20:11.232 | INFO     | __main__:<module>:34 - Total Tokens: 4140


Question: What is the weather in Gainesville

<answer>
The weather in Gainesville is:

- Temperature: 69.38 F
- Conditions: Clear sky
</answer>